In [11]:
os.getcwd()

'c:\\Users\\devan\\Downloads\\SAM2_Eval\\SAM2_Eval_Surgical_Datasets'

In [12]:
base_dataset_path = os.path.join(os.getcwd(), '../Datasets')  # Use the current working directory
base_dataset_path

'c:\\Users\\devan\\Downloads\\SAM2_Eval\\SAM2_Eval_Surgical_Datasets\\../Datasets'

In [20]:
model_cfg = r"C:\Users\devan\Downloads\SAM2_Eval\SAM2_model_and_cfg\sam2_hiera_l.yaml"
checkpoint_path = r"C:\Users\devan\Downloads\SAM2_Eval\SAM2_model_and_cfg\sam2_hiera_large.pt"
sam_model = initialize_model()

MissingConfigException: Cannot find primary config 'c:\Users\devan\Downloads\SAM2_Eval\SAM2_Eval_Surgical_Datasets\src\../SAM2_model_and_cfg/sam2_hiera_l.yaml'. Check that it's in your config search path.

Config search path:
	provider=hydra, path=pkg://hydra.conf
	provider=main, path=pkg://sam2
	provider=schema, path=structured://

In [6]:
# Imports
import os
import gc
import cv2
from src.sam2_eval_utils import (
    initialize_model,
    load_image,
    load_mask,
    select_point_prompt,
    save_predicted_mask,
    process_image
)

# Define dataset paths
base_dir = os.getcwd()  # Use the current working directory for base_dir
base_dataset_path = os.path.join(base_dir, '../Datasets')
dataset_name = "CholecSeg8k_test"
frames_path = os.path.join(base_dataset_path, "CholecSeg8k", "test", "Frames")
masks_path = os.path.join(base_dataset_path, "CholecSeg8k", "test", "Masks")
predicted_masks_path = os.path.join(base_dataset_path, "CholecSeg8k", "test", "PredictedMasks")
os.makedirs(predicted_masks_path, exist_ok=True)

# Set paths for the model configuration and checkpoint
model_cfg = r"C:\Users\devan\Downloads\SAM2_Eval\SAM2_model_and_cfg\sam2_hiera_l.yaml"
checkpoint_path = r"C:\Users\devan\Downloads\SAM2_Eval\SAM2_model_and_cfg\sam2_hiera_large.pt"
device = "cpu"

# Temporary directory for JPEG conversion
temp_jpg_folder = os.path.join(base_dir, 'temp_jpg_frames')
os.makedirs(temp_jpg_folder, exist_ok=True)

# Function to convert non-JPEG images to JPEG for temporary processing
def convert_to_jpg(image_path, temp_folder):
    """Convert an image to JPEG format and store in a temporary folder."""
    image = cv2.imread(image_path)
    jpg_path = os.path.join(temp_folder, os.path.basename(image_path).replace('.png', '.jpg'))
    cv2.imwrite(jpg_path, image)
    return jpg_path

# Run inference
def run_inference():
    # Initialize the SAM2 model
    sam_model = initialize_model()
    print(f"\nProcessing dataset: {dataset_name}")
    
    # Loop through frames and masks
    for root, _, files in os.walk(frames_path):
        for file_name in files:
            if file_name.endswith('.png') or file_name.endswith('.jpg'):
                frame_path = os.path.join(root, file_name)
                relative_path = os.path.relpath(root, frames_path)
                predicted_mask_output_folder = os.path.join(predicted_masks_path, relative_path)
                os.makedirs(predicted_mask_output_folder, exist_ok=True)
                predicted_mask_path = os.path.join(predicted_mask_output_folder, file_name)

                # Convert non-JPG frames to JPG in temp folder
                if not file_name.endswith('.jpg'):
                    frame_path = convert_to_jpg(frame_path, temp_jpg_folder)

                # Check for each mask class
                mask_found = False  # Track if any mask is found for this image
                for class_name in os.listdir(masks_path):
                    mask_folder_path = os.path.join(masks_path, class_name, relative_path)
                    mask_path = os.path.join(mask_folder_path, file_name.replace('.png', '.jpg'))
                    
                    print(f"Looking for mask in: {mask_path}")
                    
                    # Check if the mask file exists
                    if os.path.isfile(mask_path):
                        print(f"Found mask file: {mask_path}")
                        mask_found = True

                        # Load frame and mask
                        frame_image = load_image(frame_path)
                        mask_image = load_mask(mask_path)
                        if frame_image is None or mask_image is None:
                            print(f"Error loading frame or mask: {frame_path} or {mask_path}")
                            continue
                        
                        # Initialize SAM2 inference state and process image
                        inference_state = sam_model.init_state(os.path.join(frames_path, relative_path))
                        process_image(sam_model, inference_state, frame_image, mask_image, predicted_mask_path, idx=0)
                        
                        # Reset SAM2 model state
                        sam_model.reset_state(inference_state)
                        gc.collect()
                        break  # Process only the first matching mask for each frame

                if not mask_found:
                    print(f"No matching mask found for frame: {frame_path}")
                
                # Remove the temporary JPG file after processing
                if not file_name.endswith('.jpg'):
                    os.remove(frame_path)

    print("Segmentation processing completed for the dataset.")
    # Clean up temporary directory after all processing
    os.rmdir(temp_jpg_folder)

# Execute the inference
run_inference()





Processing dataset: CholecSeg8k_test
Looking for mask in: c:\Users\devan\Downloads\SAM2_Eval\SAM2_Eval_Surgical_Datasets\../Datasets\CholecSeg8k\test\Masks\Abdominal Wall\video18\video18_00979\1000.jpg
Looking for mask in: c:\Users\devan\Downloads\SAM2_Eval\SAM2_Eval_Surgical_Datasets\../Datasets\CholecSeg8k\test\Masks\Fat\video18\video18_00979\1000.jpg
Looking for mask in: c:\Users\devan\Downloads\SAM2_Eval\SAM2_Eval_Surgical_Datasets\../Datasets\CholecSeg8k\test\Masks\Gall bladder\video18\video18_00979\1000.jpg
Looking for mask in: c:\Users\devan\Downloads\SAM2_Eval\SAM2_Eval_Surgical_Datasets\../Datasets\CholecSeg8k\test\Masks\Gastrointestinal Tract\video18\video18_00979\1000.jpg
Looking for mask in: c:\Users\devan\Downloads\SAM2_Eval\SAM2_Eval_Surgical_Datasets\../Datasets\CholecSeg8k\test\Masks\Grasper\video18\video18_00979\1000.jpg
Looking for mask in: c:\Users\devan\Downloads\SAM2_Eval\SAM2_Eval_Surgical_Datasets\../Datasets\CholecSeg8k\test\Masks\L-hook Electrocautery\video18\

## One random point over single organ (50 samples)

In [ ]:
import os
import cv2
import numpy as np
import random
import torch
import gc
from sam2.build_sam import build_sam2_video_predictor

# Dynamically set the device based on CPU usage only
device = "cpu"
print(f"Device set to: {device}")

# Define model configuration and checkpoint
model_cfg = r"C:\Users\devanish\Downloads\SAM2\ReactDocker\sam2_hiera_l.yaml"
checkpoint_path = r"C:\Users\devanish\Downloads\SAM2\sam2_hiera_large.pt"

# Define the main colon folder path
colon_folder_path = r"C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon"
print(f"Colon folder path: {colon_folder_path}")

# Process only the first subfolder and all images within it
subfolder_name = os.listdir(colon_folder_path)[0]
subfolder_path = os.path.join(colon_folder_path, subfolder_name)
print(f"Processing subfolder: {subfolder_name}")

frames_folder = os.path.join(subfolder_path, 'Frames')
masks_folder = os.path.join(subfolder_path, 'Masks')
predicted_masks_folder = os.path.join(subfolder_path, 'PredictedMask')
os.makedirs(predicted_masks_folder, exist_ok=True)
print(f"Predicted masks folder created at: {predicted_masks_folder}")

# Process all images in the frames folder
image_names = sorted(os.listdir(frames_folder))
for idx, image_name in enumerate(image_names):
    print(f"\nProcessing image {idx + 1}: {image_name}")
    
    if image_name.endswith('.png'):
        image_path = os.path.join(frames_folder, image_name)
        mask_path = os.path.join(masks_folder, image_name)
        print(f"Image path: {image_path}")
        print(f"Mask path: {mask_path}")

        # Load the image and mask
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error: Failed to load image at {image_path}")
            continue
        print("Image loaded successfully.")

        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            print(f"Error: Failed to load mask at {mask_path}")
            continue
        print("Mask loaded successfully.")

        # Create a temporary JPEG folder for each image to prevent residual data
        jpeg_folder = os.path.join(subfolder_path, f"JPEG_Frames_{idx}")
        os.makedirs(jpeg_folder, exist_ok=True)

        # Save the image as a JPEG in the temporary JPEG folder
        jpeg_image_path = os.path.join(jpeg_folder, f"{idx:05d}.jpg")
        success = cv2.imwrite(jpeg_image_path, image)
        if not success:
            print(f"Error: Failed to save JPEG image at {jpeg_image_path}")
            continue
        print(f"JPEG image successfully saved at: {jpeg_image_path}")

        # Select a random point with value 255 from the mask as the prompt coordinate
        coords = np.column_stack(np.where(mask == 255))
        if coords.size == 0:
            print(f"No valid prompt point found for mask: {image_name}")
            continue
        prompt_coord = random.choice(coords)
        print(f"Selected prompt coordinate: {prompt_coord}")

        # Instantiate the SAM2 model to ensure a fresh inference state
        sam_model = build_sam2_video_predictor(
            config_file=model_cfg,
            ckpt_path=checkpoint_path,
            device=device
        )
        inference_state = sam_model.init_state(jpeg_folder)
        print("SAM2 inference state initialized.")

        # Add the prompt and run inference, getting the output mask logits
        obj_id = idx + 1  # Unique ID for each object (image)
        frame_idx = 0
        points = [[prompt_coord[1], prompt_coord[0]]]  # X, Y coordinate
        labels = [1]  # Label set to 1
        print(f"Points: {points}, Labels: {labels}")

        # Run add_new_points_or_box and get the out_mask_logits
        _, out_obj_ids, out_mask_logits = sam_model.add_new_points_or_box(
            inference_state=inference_state,
            frame_idx=frame_idx,
            obj_id=obj_id,
            points=points,
            labels=labels,
        )
        print("Inference completed for the frame.")

        # Check and save the output mask logits
        if out_mask_logits is not None:
            predicted_mask = (out_mask_logits[0].squeeze().cpu().numpy() > 0.5).astype(np.uint8) * 255
            predicted_mask_path = os.path.join(predicted_masks_folder, f"{idx:05d}.png")
            
            # Convert to BGR if necessary for saving
            predicted_mask_bgr = cv2.cvtColor(predicted_mask, cv2.COLOR_GRAY2BGR)
            success = cv2.imwrite(predicted_mask_path, predicted_mask_bgr)
            if success:
                print(f"Predicted mask successfully saved at: {predicted_mask_path}")
            else:
                print("Error: cv2.imwrite failed to save the predicted mask.")

        # Clean up by resetting SAM inference state and deleting the JPEG folder
        sam_model.reset_state(inference_state)
        del sam_model  # Explicitly delete the model
        gc.collect()  # Run garbage collection to free memory
        print("SAM2 inference state and model reset.")

        # Remove temporary JPEG folder for the current image to ensure clean state
        for filename in os.listdir(jpeg_folder):
            file_path = os.path.join(jpeg_folder, filename)
            os.remove(file_path)
        os.rmdir(jpeg_folder)  # Delete the folder itself

print("Inference complete for all images in the subfolder.")


Device set to: cpu
Colon folder path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon
Processing subfolder: 05
Predicted masks folder created at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\PredictedMask

Processing image 1: 00.png.png
Image path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Frames\00.png.png
Mask path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Masks\00.png.png
Image loaded successfully.
Mask loaded successfully.
JPEG image successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\JPEG_Frames_0\00000.jpg
Selected prompt coordinate: [ 453 1179]


frame loading (JPEG): 100%|██████████| 1/1 [00:00<00:00, 27.37it/s]


SAM2 inference state initialized.
Points: [[1179, 453]], Labels: [1]
Inference completed for the frame.
Predicted mask successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\PredictedMask\00000.png
SAM2 inference state and model reset.

Processing image 2: 01.png.png
Image path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Frames\01.png.png
Mask path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Masks\01.png.png
Image loaded successfully.
Mask loaded successfully.
JPEG image successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\JPEG_Frames_1\00001.jpg
Selected prompt coordinate: [150 693]


frame loading (JPEG): 100%|██████████| 1/1 [00:00<00:00, 26.68it/s]


SAM2 inference state initialized.
Points: [[693, 150]], Labels: [1]
Inference completed for the frame.
Predicted mask successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\PredictedMask\00001.png
SAM2 inference state and model reset.

Processing image 3: 02.png.png
Image path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Frames\02.png.png
Mask path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Masks\02.png.png
Image loaded successfully.
Mask loaded successfully.
JPEG image successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\JPEG_Frames_2\00002.jpg
Selected prompt coordinate: [158 330]


...

SAM2 inference state initialized.
Points: [[235, 675]], Labels: [1]
Inference completed for the frame.
Predicted mask successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\PredictedMask\00055.png
SAM2 inference state and model reset.

Processing image 57: 56.png.png
Image path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Frames\56.png.png
Mask path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Masks\56.png.png
Image loaded successfully.
Mask loaded successfully.
JPEG image successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\JPEG_Frames_56\00056.jpg
Selected prompt coordinate: [628 665]


frame loading (JPEG): 100%|██████████| 1/1 [00:00<00:00, 24.86it/s]


SAM2 inference state initialized.
Points: [[665, 628]], Labels: [1]
Inference completed for the frame.
Predicted mask successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\PredictedMask\00056.png
SAM2 inference state and model reset.

Processing image 58: 57.png.png
Image path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Frames\57.png.png
Mask path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Masks\57.png.png
Image loaded successfully.
Mask loaded successfully.
JPEG image successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\JPEG_Frames_57\00057.jpg
Selected prompt coordinate: [802  80]


frame loading (JPEG): 100%|██████████| 1/1 [00:00<00:00, 27.15it/s]


SAM2 inference state initialized.
Points: [[80, 802]], Labels: [1]
Inference completed for the frame.
Predicted mask successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\PredictedMask\00057.png
SAM2 inference state and model reset.
Inference complete for all images in the subfolder.


## Two random points over single organ (50 samples)

In [2]:
import os
import cv2
import numpy as np
import random
import torch
import gc
from sam2.build_sam import build_sam2_video_predictor

# Dynamically set the device based on CPU usage only
device = "cpu"
print(f"Device set to: {device}")

# Define model configuration and checkpoint
model_cfg = r"C:\Users\devanish\Downloads\SAM2\ReactDocker\sam2_hiera_l.yaml"
checkpoint_path = r"C:\Users\devanish\Downloads\SAM2\sam2_hiera_large.pt"

# Define the main colon folder path
colon_folder_path = r"C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon"
print(f"Colon folder path: {colon_folder_path}")

# Process only the first subfolder and all images within it
subfolder_name = os.listdir(colon_folder_path)[0]
subfolder_path = os.path.join(colon_folder_path, subfolder_name)
print(f"Processing subfolder: {subfolder_name}")

frames_folder = os.path.join(subfolder_path, 'Frames')
masks_folder = os.path.join(subfolder_path, 'Masks')
predicted_masks_folder = os.path.join(subfolder_path, 'PredictedMask')
os.makedirs(predicted_masks_folder, exist_ok=True)
print(f"Predicted masks folder created at: {predicted_masks_folder}")

# Process all images in the frames folder
image_names = sorted(os.listdir(frames_folder))
for idx, image_name in enumerate(image_names):
    print(f"\nProcessing image {idx + 1}: {image_name}")
    
    if image_name.endswith('.png'):
        image_path = os.path.join(frames_folder, image_name)
        mask_path = os.path.join(masks_folder, image_name)
        print(f"Image path: {image_path}")
        print(f"Mask path: {mask_path}")

        # Load the image and mask
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error: Failed to load image at {image_path}")
            continue
        print("Image loaded successfully.")

        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            print(f"Error: Failed to load mask at {mask_path}")
            continue
        print("Mask loaded successfully.")

        # Create a temporary JPEG folder for each image to prevent residual data
        jpeg_folder = os.path.join(subfolder_path, f"JPEG_Frames_{idx}")
        os.makedirs(jpeg_folder, exist_ok=True)

        # Save the image as a JPEG in the temporary JPEG folder
        jpeg_image_path = os.path.join(jpeg_folder, f"{idx:05d}.jpg")
        success = cv2.imwrite(jpeg_image_path, image)
        if not success:
            print(f"Error: Failed to save JPEG image at {jpeg_image_path}")
            continue
        print(f"JPEG image successfully saved at: {jpeg_image_path}")

        # Select two random points with value 255 from the mask as prompt coordinates
        coords = np.column_stack(np.where(mask == 255))
        if coords.size < 2:
            print(f"Insufficient valid prompt points found for mask: {image_name}")
            continue
        prompt_coords = random.sample(list(coords), 2)
        print(f"Selected prompt coordinates: {prompt_coords}")

        # Instantiate the SAM2 model to ensure a fresh inference state
        sam_model = build_sam2_video_predictor(
            config_file=model_cfg,
            ckpt_path=checkpoint_path,
            device=device
        )
        inference_state = sam_model.init_state(jpeg_folder)
        print("SAM2 inference state initialized.")

        # Add the two prompts and run inference, getting the output mask logits
        obj_id = idx + 1  # Unique ID for each object (image)
        frame_idx = 0
        points = [[coord[1], coord[0]] for coord in prompt_coords]  # Convert Y, X to X, Y
        labels = [1, 1]  # Set labels to 1 for both points
        print(f"Points: {points}, Labels: {labels}")

        # Run add_new_points_or_box and get the out_mask_logits
        _, out_obj_ids, out_mask_logits = sam_model.add_new_points_or_box(
            inference_state=inference_state,
            frame_idx=frame_idx,
            obj_id=obj_id,
            points=points,
            labels=labels,
        )
        print("Inference completed for the frame.")

        # Check and save the output mask logits
        if out_mask_logits is not None:
            predicted_mask = (out_mask_logits[0].squeeze().cpu().numpy() > 0.5).astype(np.uint8) * 255
            predicted_mask_path = os.path.join(predicted_masks_folder, f"{idx:05d}.png")
            
            # Convert to BGR if necessary for saving
            predicted_mask_bgr = cv2.cvtColor(predicted_mask, cv2.COLOR_GRAY2BGR)
            success = cv2.imwrite(predicted_mask_path, predicted_mask_bgr)
            if success:
                print(f"Predicted mask successfully saved at: {predicted_mask_path}")
            else:
                print("Error: cv2.imwrite failed to save the predicted mask.")

        # Clean up by resetting SAM inference state and deleting the JPEG folder
        sam_model.reset_state(inference_state)
        del sam_model  # Explicitly delete the model
        gc.collect()  # Run garbage collection to free memory
        print("SAM2 inference state and model reset.")

        # Remove temporary JPEG folder for the current image to ensure clean state
        for filename in os.listdir(jpeg_folder):
            file_path = os.path.join(jpeg_folder, filename)
            os.remove(file_path)
        os.rmdir(jpeg_folder)  # Delete the folder itself

print("Inference complete for all images in the subfolder.")


Device set to: cpu
Colon folder path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon
Processing subfolder: 05
Predicted masks folder created at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\PredictedMask

Processing image 1: 00000.png
Image path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Frames\00000.png
Mask path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Masks\00000.png
Image loaded successfully.
Mask loaded successfully.
JPEG image successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\JPEG_Frames_0\00000.jpg
Selected prompt coordinates: [array([269, 962], dtype=int64), array([297, 990], dtype=int64)]


frame loading (JPEG): 100%|██████████| 1/1 [00:00<00:00, 11.05it/s]


SAM2 inference state initialized.
Points: [[962, 269], [990, 297]], Labels: [1, 1]
Inference completed for the frame.
Predicted mask successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\PredictedMask\00000.png


c:\Users\devanish\AppData\Local\anaconda3\Lib\site-packages\sam2\sam2_video_predictor.py:873: UserWarning: cannot import name '_C' from 'sam2' (c:\Users\devanish\AppData\Local\anaconda3\Lib\site-packages\sam2\__init__.py)

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/segment-anything-2/blob/main/INSTALL.md).
  pred_masks_gpu = fill_holes_in_mask_scores(


SAM2 inference state and model reset.

Processing image 2: 00001.png
Image path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Frames\00001.png
Mask path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Masks\00001.png
Image loaded successfully.
Mask loaded successfully.
JPEG image successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\JPEG_Frames_1\00001.jpg
Selected prompt coordinates: [array([202, 626], dtype=int64), array([347, 260], dtype=int64)]


...

SAM2 inference state initialized.
Points: [[183, 911], [184, 755]], Labels: [1, 1]
Inference completed for the frame.
Predicted mask successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\PredictedMask\00055.png
SAM2 inference state and model reset.

Processing image 57: 00056.png
Image path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Frames\00056.png
Mask path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Masks\00056.png
Image loaded successfully.
Mask loaded successfully.
JPEG image successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\JPEG_Frames_56\00056.jpg
Selected prompt coordinates: [array([711, 727], dtype=int64), array([952, 507], dtype=int64)]


frame loading (JPEG): 100%|██████████| 1/1 [00:00<00:00, 24.19it/s]


SAM2 inference state initialized.
Points: [[727, 711], [507, 952]], Labels: [1, 1]
Inference completed for the frame.
Predicted mask successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\PredictedMask\00056.png
SAM2 inference state and model reset.

Processing image 58: 00057.png
Image path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Frames\00057.png
Mask path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Masks\00057.png
Image loaded successfully.
Mask loaded successfully.
JPEG image successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\JPEG_Frames_57\00057.jpg
Selected prompt coordinates: [array([923, 468], dtype=int64), array([749, 461], dtype=int64)]


frame loading (JPEG): 100%|██████████| 1/1 [00:00<00:00, 25.13it/s]


SAM2 inference state initialized.
Points: [[468, 923], [461, 749]], Labels: [1, 1]
Inference completed for the frame.
Predicted mask successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\PredictedMask\00057.png
SAM2 inference state and model reset.
Inference complete for all images in the subfolder.


## Three Random points over single organ (50 samples)

In [3]:
import os
import cv2
import numpy as np
import random
import torch
import gc
from sam2.build_sam import build_sam2_video_predictor

# Dynamically set the device based on CPU usage only
device = "cpu"
print(f"Device set to: {device}")

# Define model configuration and checkpoint
model_cfg = r"C:\Users\devanish\Downloads\SAM2\ReactDocker\sam2_hiera_l.yaml"
checkpoint_path = r"C:\Users\devanish\Downloads\SAM2\sam2_hiera_large.pt"

# Define the main colon folder path
colon_folder_path = r"C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon"
print(f"Colon folder path: {colon_folder_path}")

# Process only the first subfolder and all images within it
subfolder_name = os.listdir(colon_folder_path)[0]
subfolder_path = os.path.join(colon_folder_path, subfolder_name)
print(f"Processing subfolder: {subfolder_name}")

frames_folder = os.path.join(subfolder_path, 'Frames')
masks_folder = os.path.join(subfolder_path, 'Masks')
predicted_masks_folder = os.path.join(subfolder_path, 'PredictedMask')
os.makedirs(predicted_masks_folder, exist_ok=True)
print(f"Predicted masks folder created at: {predicted_masks_folder}")

# Process all images in the frames folder
image_names = sorted(os.listdir(frames_folder))
for idx, image_name in enumerate(image_names):
    print(f"\nProcessing image {idx + 1}: {image_name}")
    
    if image_name.endswith('.png'):
        image_path = os.path.join(frames_folder, image_name)
        mask_path = os.path.join(masks_folder, image_name)
        print(f"Image path: {image_path}")
        print(f"Mask path: {mask_path}")

        # Load the image and mask
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error: Failed to load image at {image_path}")
            continue
        print("Image loaded successfully.")

        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            print(f"Error: Failed to load mask at {mask_path}")
            continue
        print("Mask loaded successfully.")

        # Create a temporary JPEG folder for each image to prevent residual data
        jpeg_folder = os.path.join(subfolder_path, f"JPEG_Frames_{idx}")
        os.makedirs(jpeg_folder, exist_ok=True)

        # Save the image as a JPEG in the temporary JPEG folder
        jpeg_image_path = os.path.join(jpeg_folder, f"{idx:05d}.jpg")
        success = cv2.imwrite(jpeg_image_path, image)
        if not success:
            print(f"Error: Failed to save JPEG image at {jpeg_image_path}")
            continue
        print(f"JPEG image successfully saved at: {jpeg_image_path}")

        # Select three random points with value 255 from the mask as prompt coordinates
        coords = np.column_stack(np.where(mask == 255))
        if coords.size < 3:
            print(f"Insufficient valid prompt points found for mask: {image_name}")
            continue
        prompt_coords = random.sample(list(coords), 3)
        print(f"Selected prompt coordinates: {prompt_coords}")

        # Instantiate the SAM2 model to ensure a fresh inference state
        sam_model = build_sam2_video_predictor(
            config_file=model_cfg,
            ckpt_path=checkpoint_path,
            device=device
        )
        inference_state = sam_model.init_state(jpeg_folder)
        print("SAM2 inference state initialized.")

        # Add the three prompts and run inference, getting the output mask logits
        obj_id = idx + 1  # Unique ID for each object (image)
        frame_idx = 0
        points = [[coord[1], coord[0]] for coord in prompt_coords]  # Convert Y, X to X, Y
        labels = [1, 1, 1]  # Set labels to 1 for all points
        print(f"Points: {points}, Labels: {labels}")

        # Run add_new_points_or_box and get the out_mask_logits
        _, out_obj_ids, out_mask_logits = sam_model.add_new_points_or_box(
            inference_state=inference_state,
            frame_idx=frame_idx,
            obj_id=obj_id,
            points=points,
            labels=labels,
        )
        print("Inference completed for the frame.")

        # Check and save the output mask logits
        if out_mask_logits is not None:
            predicted_mask = (out_mask_logits[0].squeeze().cpu().numpy() > 0.5).astype(np.uint8) * 255
            predicted_mask_path = os.path.join(predicted_masks_folder, f"{idx:05d}.png")
            
            # Convert to BGR if necessary for saving
            predicted_mask_bgr = cv2.cvtColor(predicted_mask, cv2.COLOR_GRAY2BGR)
            success = cv2.imwrite(predicted_mask_path, predicted_mask_bgr)
            if success:
                print(f"Predicted mask successfully saved at: {predicted_mask_path}")
            else:
                print("Error: cv2.imwrite failed to save the predicted mask.")

        # Clean up by resetting SAM inference state and deleting the JPEG folder
        sam_model.reset_state(inference_state)
        del sam_model  # Explicitly delete the model
        gc.collect()  # Run garbage collection to free memory
        print("SAM2 inference state and model reset.")

        # Remove temporary JPEG folder for the current image to ensure clean state
        for filename in os.listdir(jpeg_folder):
            file_path = os.path.join(jpeg_folder, filename)
            os.remove(file_path)
        os.rmdir(jpeg_folder)  # Delete the folder itself

print("Inference complete for all images in the subfolder.")


Device set to: cpu
Colon folder path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon
Processing subfolder: 05
Predicted masks folder created at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\PredictedMask

Processing image 1: 00000.png
Image path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Frames\00000.png
Mask path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Masks\00000.png
Image loaded successfully.
Mask loaded successfully.
JPEG image successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\JPEG_Frames_0\00000.jpg
Selected prompt coordinates: [array([ 285, 1006], dtype=int64), array([ 322, 1225], dtype=int64), array([ 400, 1164], dtype=int64)]


frame loading (JPEG): 100%|██████████| 1/1 [00:00<00:00, 22.73it/s]


SAM2 inference state initialized.
Points: [[1006, 285], [1225, 322], [1164, 400]], Labels: [1, 1, 1]
Inference completed for the frame.
Predicted mask successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\PredictedMask\00000.png
SAM2 inference state and model reset.

Processing image 2: 00001.png
Image path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Frames\00001.png
Mask path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Masks\00001.png
Image loaded successfully.
Mask loaded successfully.
JPEG image successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\JPEG_Frames_1\00001.jpg
Selected prompt coordinates: [array([369, 124], dtype=int64), array([112, 721], dtype=int64), array([385, 283], dtype=int64)]


frame loading (JPEG): 100%|██████████| 1/1 [00:00<00:00,  2.86it/s]


SAM2 inference state initialized.
Points: [[124, 369], [721, 112], [283, 385]], Labels: [1, 1, 1]
Inference completed for the frame.
Predicted mask successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\PredictedMask\00001.png
SAM2 inference state and model reset.

Processing image 3: 00002.png
Image path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Frames\00002.png
Mask path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Masks\00002.png
Image loaded successfully.
Mask loaded successfully.
JPEG image successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\JPEG_Frames_2\00002.jpg
Selected prompt coordinates: [array([186, 395], dtype=int64), array([365, 549], dtype=int64), array([293, 473], dtype=int64)]


...

SAM2 inference state initialized.
Points: [[381, 647], [70, 948], [270, 715]], Labels: [1, 1, 1]
Inference completed for the frame.
Predicted mask successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\PredictedMask\00055.png
SAM2 inference state and model reset.

Processing image 57: 00056.png
Image path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Frames\00056.png
Mask path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Masks\00056.png
Image loaded successfully.
Mask loaded successfully.
JPEG image successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\JPEG_Frames_56\00056.jpg
Selected prompt coordinates: [array([873, 334], dtype=int64), array([770, 477], dtype=int64), array([647, 663], dtype=int64)]


frame loading (JPEG): 100%|██████████| 1/1 [00:00<00:00, 26.32it/s]


SAM2 inference state initialized.
Points: [[334, 873], [477, 770], [663, 647]], Labels: [1, 1, 1]
Inference completed for the frame.
Predicted mask successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\PredictedMask\00056.png
SAM2 inference state and model reset.

Processing image 58: 00057.png
Image path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Frames\00057.png
Mask path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Masks\00057.png
Image loaded successfully.
Mask loaded successfully.
JPEG image successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\JPEG_Frames_57\00057.jpg
Selected prompt coordinates: [array([658, 489], dtype=int64), array([942, 497], dtype=int64), array([866, 516], dtype=int64)]


frame loading (JPEG): 100%|██████████| 1/1 [00:00<00:00, 25.00it/s]


SAM2 inference state initialized.
Points: [[489, 658], [497, 942], [516, 866]], Labels: [1, 1, 1]
Inference completed for the frame.
Predicted mask successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\PredictedMask\00057.png
SAM2 inference state and model reset.
Inference complete for all images in the subfolder.


## Four points over a single organ (50 samples)

In [4]:
import os
import cv2
import numpy as np
import random
import torch
import gc
from sam2.build_sam import build_sam2_video_predictor

# Dynamically set the device based on CPU usage only
device = "cpu"
print(f"Device set to: {device}")

# Define model configuration and checkpoint
model_cfg = r"C:\Users\devanish\Downloads\SAM2\ReactDocker\sam2_hiera_l.yaml"
checkpoint_path = r"C:\Users\devanish\Downloads\SAM2\sam2_hiera_large.pt"

# Define the main colon folder path
colon_folder_path = r"C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon"
print(f"Colon folder path: {colon_folder_path}")

# Process only the first subfolder and all images within it
subfolder_name = os.listdir(colon_folder_path)[0]
subfolder_path = os.path.join(colon_folder_path, subfolder_name)
print(f"Processing subfolder: {subfolder_name}")

frames_folder = os.path.join(subfolder_path, 'Frames')
masks_folder = os.path.join(subfolder_path, 'Masks')
predicted_masks_folder = os.path.join(subfolder_path, 'PredictedMask')
os.makedirs(predicted_masks_folder, exist_ok=True)
print(f"Predicted masks folder created at: {predicted_masks_folder}")

# Process all images in the frames folder
image_names = sorted(os.listdir(frames_folder))
for idx, image_name in enumerate(image_names):
    print(f"\nProcessing image {idx + 1}: {image_name}")
    
    if image_name.endswith('.png'):
        image_path = os.path.join(frames_folder, image_name)
        mask_path = os.path.join(masks_folder, image_name)
        print(f"Image path: {image_path}")
        print(f"Mask path: {mask_path}")

        # Load the image and mask
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error: Failed to load image at {image_path}")
            continue
        print("Image loaded successfully.")

        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            print(f"Error: Failed to load mask at {mask_path}")
            continue
        print("Mask loaded successfully.")

        # Create a temporary JPEG folder for each image to prevent residual data
        jpeg_folder = os.path.join(subfolder_path, f"JPEG_Frames_{idx}")
        os.makedirs(jpeg_folder, exist_ok=True)

        # Save the image as a JPEG in the temporary JPEG folder
        jpeg_image_path = os.path.join(jpeg_folder, f"{idx:05d}.jpg")
        success = cv2.imwrite(jpeg_image_path, image)
        if not success:
            print(f"Error: Failed to save JPEG image at {jpeg_image_path}")
            continue
        print(f"JPEG image successfully saved at: {jpeg_image_path}")

        # Select four random points with value 255 from the mask as prompt coordinates
        coords = np.column_stack(np.where(mask == 255))
        if coords.size < 4:
            print(f"Insufficient valid prompt points found for mask: {image_name}")
            continue
        prompt_coords = random.sample(list(coords), 4)
        print(f"Selected prompt coordinates: {prompt_coords}")

        # Instantiate the SAM2 model to ensure a fresh inference state
        sam_model = build_sam2_video_predictor(
            config_file=model_cfg,
            ckpt_path=checkpoint_path,
            device=device
        )
        inference_state = sam_model.init_state(jpeg_folder)
        print("SAM2 inference state initialized.")

        # Add the four prompts and run inference, getting the output mask logits
        obj_id = idx + 1  # Unique ID for each object (image)
        frame_idx = 0
        points = [[coord[1], coord[0]] for coord in prompt_coords]  # Convert Y, X to X, Y
        labels = [1, 1, 1, 1]  # Set labels to 1 for all points
        print(f"Points: {points}, Labels: {labels}")

        # Run add_new_points_or_box and get the out_mask_logits
        _, out_obj_ids, out_mask_logits = sam_model.add_new_points_or_box(
            inference_state=inference_state,
            frame_idx=frame_idx,
            obj_id=obj_id,
            points=points,
            labels=labels,
        )
        print("Inference completed for the frame.")

        # Check and save the output mask logits
        if out_mask_logits is not None:
            predicted_mask = (out_mask_logits[0].squeeze().cpu().numpy() > 0.5).astype(np.uint8) * 255
            predicted_mask_path = os.path.join(predicted_masks_folder, f"{idx:05d}.png")
            
            # Convert to BGR if necessary for saving
            predicted_mask_bgr = cv2.cvtColor(predicted_mask, cv2.COLOR_GRAY2BGR)
            success = cv2.imwrite(predicted_mask_path, predicted_mask_bgr)
            if success:
                print(f"Predicted mask successfully saved at: {predicted_mask_path}")
            else:
                print("Error: cv2.imwrite failed to save the predicted mask.")

        # Clean up by resetting SAM inference state and deleting the JPEG folder
        sam_model.reset_state(inference_state)
        del sam_model  # Explicitly delete the model
        gc.collect()  # Run garbage collection to free memory
        print("SAM2 inference state and model reset.")

        # Remove temporary JPEG folder for the current image to ensure clean state
        for filename in os.listdir(jpeg_folder):
            file_path = os.path.join(jpeg_folder, filename)
            os.remove(file_path)
        os.rmdir(jpeg_folder)  # Delete the folder itself

print("Inference complete for all images in the subfolder.")


Device set to: cpu
Colon folder path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon
Processing subfolder: 05
Predicted masks folder created at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\PredictedMask

Processing image 1: 00000.png
Image path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Frames\00000.png
Mask path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Masks\00000.png
Image loaded successfully.
Mask loaded successfully.
JPEG image successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\JPEG_Frames_0\00000.jpg
Selected prompt coordinates: [array([ 364, 1127], dtype=int64), array([ 333, 1061], dtype=int64), array([ 468, 1248], dtype=int64), array([ 438, 1239], dtype=int64)]


frame loading (JPEG): 100%|██████████| 1/1 [00:00<00:00, 24.39it/s]


SAM2 inference state initialized.
Points: [[1127, 364], [1061, 333], [1248, 468], [1239, 438]], Labels: [1, 1, 1, 1]
Inference completed for the frame.
Predicted mask successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\PredictedMask\00000.png
SAM2 inference state and model reset.

Processing image 2: 00001.png
Image path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Frames\00001.png
Mask path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Masks\00001.png
Image loaded successfully.
Mask loaded successfully.
JPEG image successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\JPEG_Frames_1\00001.jpg
Selected prompt coordinates: [array([179, 641], dtype=int64), array([166, 693], dtype=int64), array([254, 821], dtype=int64), array([258, 170], dtype=int64)]


frame loading (JPEG): 100%|██████████| 1/1 [00:00<00:00, 24.39it/s]


SAM2 inference state initialized.
Points: [[641, 179], [693, 166], [821, 254], [170, 258]], Labels: [1, 1, 1, 1]
Inference completed for the frame.
Predicted mask successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\PredictedMask\00001.png
SAM2 inference state and model reset.

Processing image 3: 00002.png
Image path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Frames\00002.png
Mask path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Masks\00002.png
Image loaded successfully.
Mask loaded successfully.
JPEG image successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\JPEG_Frames_2\00002.jpg
Selected prompt coordinates: [array([248, 475], dtype=int64), array([338, 361], dtype=int64), array([153, 442], dtype=int64), array([472, 590], dtype=int64)]


...

SAM2 inference state initialized.
Points: [[384, 617], [134, 715], [109, 816], [404, 612]], Labels: [1, 1, 1, 1]
Inference completed for the frame.
Predicted mask successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\PredictedMask\00055.png
SAM2 inference state and model reset.

Processing image 57: 00056.png
Image path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Frames\00056.png
Mask path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Masks\00056.png
Image loaded successfully.
Mask loaded successfully.
JPEG image successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\JPEG_Frames_56\00056.jpg
Selected prompt coordinates: [array([802, 442], dtype=int64), array([906, 572], dtype=int64), array([560, 801], dtype=int64), array([807, 528], dtype=int64)]


frame loading (JPEG): 100%|██████████| 1/1 [00:00<00:00, 25.00it/s]


SAM2 inference state initialized.
Points: [[442, 802], [572, 906], [801, 560], [528, 807]], Labels: [1, 1, 1, 1]
Inference completed for the frame.
Predicted mask successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\PredictedMask\00056.png
SAM2 inference state and model reset.

Processing image 58: 00057.png
Image path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Frames\00057.png
Mask path: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\Masks\00057.png
Image loaded successfully.
Mask loaded successfully.
JPEG image successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\JPEG_Frames_57\00057.jpg
Selected prompt coordinates: [array([678, 407], dtype=int64), array([620, 448], dtype=int64), array([447, 557], dtype=int64), array([673, 554], dtype=int64)]


frame loading (JPEG): 100%|██████████| 1/1 [00:00<00:00, 12.50it/s]


SAM2 inference state initialized.
Points: [[407, 678], [448, 620], [557, 447], [554, 673]], Labels: [1, 1, 1, 1]
Inference completed for the frame.
Predicted mask successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\Colon\05\PredictedMask\00057.png
SAM2 inference state and model reset.
Inference complete for all images in the subfolder.


## Two points and a negative point from the negative area

## Single point all organs (50 sample)

In [10]:
import os
import cv2
import numpy as np
import random
import torch
import gc
from sam2.build_sam import build_sam2_video_predictor

# Dynamically set the device based on CPU usage only
device = "cpu"
print(f"Device set to: {device}")

# Define model configuration and checkpoint
model_cfg = r"C:\Users\devanish\Downloads\SAM2\ReactDocker\sam2_hiera_l.yaml"
checkpoint_path = r"C:\Users\devanish\Downloads\SAM2\sam2_hiera_large.pt"

# Define the main UnitTesting folder path containing multiple organs
unit_testing_folder = r"C:\Users\devanish\Downloads\Datasets\UnitTesting"
print(f"UnitTesting folder path: {unit_testing_folder}")

# Loop through each organ folder in the UnitTesting folder
for organ_name in os.listdir(unit_testing_folder):
    organ_path = os.path.join(unit_testing_folder, organ_name)
    if not os.path.isdir(organ_path):
        continue

    # Assume there's a single subfolder inside each organ folder that contains Frames and Masks
    subfolder_name = os.listdir(organ_path)[0]
    subfolder_path = os.path.join(organ_path, subfolder_name)
    print(f"\nProcessing organ: {organ_name} in subfolder: {subfolder_name}")

    # Define paths for frames, masks, and predicted mask output for this organ
    frames_folder = os.path.join(subfolder_path, 'Frames')
    masks_folder = os.path.join(subfolder_path, 'Masks')
    predicted_masks_folder = os.path.join(subfolder_path, 'PredictedMask')
    os.makedirs(predicted_masks_folder, exist_ok=True)
    print(f"Predicted masks folder created at: {predicted_masks_folder}")

    # Process all images in the frames folder for this organ
    image_names = sorted(os.listdir(frames_folder))
    for idx, image_name in enumerate(image_names):
        print(f"\nProcessing image {idx + 1}: {image_name}")
        
        if image_name.endswith('.png'):
            image_path = os.path.join(frames_folder, image_name)
            mask_path = os.path.join(masks_folder, image_name)  # Corresponding mask with the same name
            print(f"Image path: {image_path}")
            print(f"Mask path: {mask_path}")

            # Load the image and mask
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error: Failed to load image at {image_path}")
                continue
            print("Image loaded successfully.")

            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            if mask is None:
                print(f"Error: Failed to load mask at {mask_path}")
                continue
            print("Mask loaded successfully.")

            # Create a temporary JPEG folder for each image to prevent residual data
            jpeg_folder = os.path.join(subfolder_path, f"JPEG_Frames_{idx}")
            os.makedirs(jpeg_folder, exist_ok=True)

            # Save the image as a JPEG in the temporary JPEG folder
            jpeg_image_path = os.path.join(jpeg_folder, f"{idx:05d}.jpg")
            success = cv2.imwrite(jpeg_image_path, image)
            if not success:
                print(f"Error: Failed to save JPEG image at {jpeg_image_path}")
                continue
            print(f"JPEG image successfully saved at: {jpeg_image_path}")

            # Select a random point with value 255 from the mask as the prompt coordinate
            coords = np.column_stack(np.where(mask == 255))
            if coords.size == 0:
                print(f"No valid prompt point found for mask: {image_name}")
                continue
            prompt_coord = random.choice(coords)
            print(f"Selected prompt coordinate: {prompt_coord}")

            # Instantiate the SAM2 model to ensure a fresh inference state
            sam_model = build_sam2_video_predictor(
                config_file=model_cfg,
                ckpt_path=checkpoint_path,
                device=device
            )
            inference_state = sam_model.init_state(jpeg_folder)
            print("SAM2 inference state initialized.")

            # Add the prompt and run inference, getting the output mask logits
            obj_id = idx + 1  # Unique ID for each object (image)
            frame_idx = 0
            points = [[prompt_coord[1], prompt_coord[0]]]  # X, Y coordinate
            labels = [1]  # Label set to 1
            print(f"Points: {points}, Labels: {labels}")

            # Run add_new_points_or_box and get the out_mask_logits
            _, out_obj_ids, out_mask_logits = sam_model.add_new_points_or_box(
                inference_state=inference_state,
                frame_idx=frame_idx,
                obj_id=obj_id,
                points=points,
                labels=labels,
            )
            print("Inference completed for the frame.")

            # Check and save the output mask logits
            if out_mask_logits is not None:
                predicted_mask = (out_mask_logits[0].squeeze().cpu().numpy() > 0.5).astype(np.uint8) * 255
                predicted_mask_path = os.path.join(predicted_masks_folder, f"{idx:05d}.png")
                
                # Convert to BGR if necessary for saving
                predicted_mask_bgr = cv2.cvtColor(predicted_mask, cv2.COLOR_GRAY2BGR)
                success = cv2.imwrite(predicted_mask_path, predicted_mask_bgr)
                if success:
                    print(f"Predicted mask successfully saved at: {predicted_mask_path}")
                else:
                    print("Error: cv2.imwrite failed to save the predicted mask.")

            # Clean up by resetting SAM inference state and deleting the JPEG folder
            sam_model.reset_state(inference_state)
            del sam_model  # Explicitly delete the model
            gc.collect()  # Run garbage collection to free memory
            print("SAM2 inference state and model reset.")

            # Remove temporary JPEG folder for the current image to ensure clean state
            for filename in os.listdir(jpeg_folder):
                file_path = os.path.join(jpeg_folder, filename)
                os.remove(file_path)
            os.rmdir(jpeg_folder)  # Delete the folder itself

    print(f"Inference complete for all images in the organ: {organ_name}")

print("Inference complete for all organs.")


Device set to: cpu
UnitTesting folder path: C:\Users\devanish\Downloads\Datasets\UnitTesting

Processing organ: ureter in subfolder: 01
Predicted masks folder created at: C:\Users\devanish\Downloads\Datasets\UnitTesting\ureter\01\PredictedMask

Processing image 1: 00000.png
Image path: C:\Users\devanish\Downloads\Datasets\UnitTesting\ureter\01\Frames\00000.png
Mask path: C:\Users\devanish\Downloads\Datasets\UnitTesting\ureter\01\Masks\00000.png
Image loaded successfully.
Mask loaded successfully.
JPEG image successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\ureter\01\JPEG_Frames_0\00000.jpg
Selected prompt coordinate: [339 490]


frame loading (JPEG): 100%|██████████| 1/1 [00:00<00:00, 19.98it/s]


SAM2 inference state initialized.
Points: [[490, 339]], Labels: [1]
Inference completed for the frame.
Predicted mask successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\ureter\01\PredictedMask\00000.png
SAM2 inference state and model reset.

Processing image 2: 00001.png
Image path: C:\Users\devanish\Downloads\Datasets\UnitTesting\ureter\01\Frames\00001.png
Mask path: C:\Users\devanish\Downloads\Datasets\UnitTesting\ureter\01\Masks\00001.png
Image loaded successfully.
Mask loaded successfully.
JPEG image successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\ureter\01\JPEG_Frames_1\00001.jpg
Selected prompt coordinate: [327 530]


frame loading (JPEG): 100%|██████████| 1/1 [00:00<00:00, 20.21it/s]


SAM2 inference state initialized.
Points: [[530, 327]], Labels: [1]
Inference completed for the frame.
Predicted mask successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\ureter\01\PredictedMask\00001.png
SAM2 inference state and model reset.

Processing image 3: 00002.png
Image path: C:\Users\devanish\Downloads\Datasets\UnitTesting\ureter\01\Frames\00002.png
Mask path: C:\Users\devanish\Downloads\Datasets\UnitTesting\ureter\01\Masks\00002.png
Image loaded successfully.
Mask loaded successfully.
JPEG image successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\ureter\01\JPEG_Frames_2\00002.jpg
Selected prompt coordinate: [564 264]


...

SAM2 inference state initialized.
Points: [[385, 199]], Labels: [1]
Inference completed for the frame.
Predicted mask successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\vesicular_glands\03\PredictedMask\00052.png
SAM2 inference state and model reset.

Processing image 54: 00053.png
Image path: C:\Users\devanish\Downloads\Datasets\UnitTesting\vesicular_glands\03\Frames\00053.png
Mask path: C:\Users\devanish\Downloads\Datasets\UnitTesting\vesicular_glands\03\Masks\00053.png
Image loaded successfully.
Mask loaded successfully.
JPEG image successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\vesicular_glands\03\JPEG_Frames_53\00053.jpg
Selected prompt coordinate: [504 471]


frame loading (JPEG): 100%|██████████| 1/1 [00:00<00:00, 24.46it/s]


SAM2 inference state initialized.
Points: [[471, 504]], Labels: [1]
Inference completed for the frame.
Predicted mask successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\vesicular_glands\03\PredictedMask\00053.png
SAM2 inference state and model reset.

Processing image 55: 00054.png
Image path: C:\Users\devanish\Downloads\Datasets\UnitTesting\vesicular_glands\03\Frames\00054.png
Mask path: C:\Users\devanish\Downloads\Datasets\UnitTesting\vesicular_glands\03\Masks\00054.png
Image loaded successfully.
Mask loaded successfully.
JPEG image successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\vesicular_glands\03\JPEG_Frames_54\00054.jpg
Selected prompt coordinate: [269 293]


frame loading (JPEG): 100%|██████████| 1/1 [00:00<00:00, 25.15it/s]


SAM2 inference state initialized.
Points: [[293, 269]], Labels: [1]
Inference completed for the frame.
Predicted mask successfully saved at: C:\Users\devanish\Downloads\Datasets\UnitTesting\vesicular_glands\03\PredictedMask\00054.png
SAM2 inference state and model reset.
Inference complete for all images in the organ: vesicular_glands
Inference complete for all organs.


## Two points and all organs (50 samples)

## Three points and all organs (50 samples)

## Two points with another negative point and all organs (50 samples)